<a href="https://colab.research.google.com/github/AlekSSSandraJ/readnew/blob/main/NLP_CW_Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TASK 1
TO DO:
1. Pre-process the Dataset [5 marks]. Develop code to clean and pre-process the dataset. Look at the dictionary to ensure you’ve cleaned the data well.
2. Latent Semantic Indexing (LSI) and Neural Information Retrieval [25 marks].
    * Develop and tune a Latent Semantic Indexing (LSI) model
    * and a Neural information retrieval model (15 marks).
    
    * Evaluate the performance of the models using suitable evaluation metrics and present the results of your evaluations using tables and graphs.
    *  Provide brief remarks on your observations (10 marks).

3. Clustering and Analysis [10 marks]. Extend your answer to Task 2 to include clustering of search results. This algorithm should take as input the top 100 search results of your chosen query and cluster them into topics. Dynamically determine the optimal number of clusters using a suitable method (5 marks). Evaluate the clustering performance of the model using suitable evaluation metrics and provide brief remarks on your evaluations. (5 marks).
4. SummariseTopicswithinClustersofSearchResults[10marks].ExtendyouranswertoTask3toincludeanapproach that takes the results from each cluster and summarises them using one abstractive and one extractive method (5 marks). Evaluate the performance of the models using suitable evaluation metrics. Note that you can use ChatGPT or another tool to create the reference summaries. Explain the findings. (5 marks).

1. PRE PROCESSING

Removing noise: Strip out unnecessary symbols, numbers, and formatting in the reviews.
Tokenization: Break down the text into individual words or tokens.
Normalization: Convert all text to lowercase, standardize terms, and perhaps employ stemming or lemmatization.
Removing stop words: Filter out common words that might not be useful in analysis.
Spell check: Correct spelling errors within the reviews.

1. Pre-Processing

  1.1 Load Datasets

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls "/content/drive/My Drive/Colab Notebooks/COP509cw/Datasets/"
Data_path = "/content/drive/My Drive/Colab Notebooks/COP509cw/Datasets/"

JewelleryReviewsLSA.csv		     JewelleryReviewsSummarisationTargets.csv  README.md
JewelleryReviewsQueryRelevantID.csv  QueryText.csv.xlsx


In [3]:
import pandas as pd

# Paths to the datasets
data_path = "/content/drive/My Drive/Colab Notebooks/COP509cw/Datasets/"

reviews_path = data_path + "JewelleryReviewsLSA.csv"
query_relevance_path = data_path + "JewelleryReviewsQueryRelevantID.csv"
summarisation_targets_path = data_path + "JewelleryReviewsSummarisationTargets.csv"
query_text_path = data_path + "QueryText.csv.xlsx"

# Loading the datasets
reviews_df = pd.read_csv(reviews_path)
query_relevance_df = pd.read_csv(query_relevance_path)
summarisation_targets_df = pd.read_csv(summarisation_targets_path)
query_text_df = pd.read_excel(query_text_path)

# Display the first few rows of the reviews dataset to check
print(reviews_df.head())


      ID                                            Reviews  Ratings
0  32767  i expect like regular size of ring,but this on...        1
1  15959  I ordered a ring that stated Toe Ring" in the ...        1
2  43515  This ring is a beautiful ring but the first sh...        1
3  46347  This ring is a beautiful ring but the first sh...        1
4  30720  This is a huge waste of money I had it for abo...        1


1.2 Cleaning Dataset

Remove Non-Alphabetic Characters:
It's useful for removing unnecessary punctuation and numbers, which typically do not contribute to sentiment analysis or other NLP tasks unless specific numeric or symbolic meanings are relevant (e.g., "5-star").

Convert to Lowercase:
This standardizes the text to ensure that the same words are recognized regardless of their place in a sentence.

Tokenize:
Convert strings into lists of tokens or words. You can do this more thoroughly by considering not just spaces but punctuation and special characters as delimiters.

Remove Stopwords:
Stopwords (common words like 'and', 'the', etc.) are usually removed because they do not contribute to the overall meaning of a sentence in the context of many analysis tasks.
Lemmatization:
This reduces words to their base or root form, which helps in consolidating different forms of a word into a single representation (e.g., "running" to "run").

In [1]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [16]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
def preprocess_text(text):
    # Adjust regex to avoid removing apostrophes in contractions and preserve hyphenated words
    text = re.sub(r"[^\w\s'-]", '', text)

    # Convert text to lowercase
    text = text.lower()

    # Tokenize text while preserving contractions
    words = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words and word not in string.punctuation]

    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    # Join words to form the cleaned text
    return ' '.join(lemmatized_words)


In [17]:
# Apply preprocessing to the Reviews column
reviews_df['Processed_Reviews'] = reviews_df['Reviews'].apply(preprocess_text)

# Check the first few processed reviews to ensure everything is as expected
print(reviews_df[['Reviews', 'Processed_Reviews']].head())


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [10]:
summarisation_targets_df['Processed_Targets'] = summarisation_targets_df['Targets'].apply(preprocess_text)

# Check the first few processed targets to ensure everything is as expected
print(summarisation_targets_df[['Targets', 'Processed_Targets']].head())

                                             Targets  \
0  I will not be buying this ring again. I will n...   
1  It was not worth returning it as you would pay...   
2  This ring is a beautiful ring but the first sh...   
3  I love the shipping pace. The ring itself is b...   
4  This ring is gorgeous. I love the ring and wou...   

                                   Processed_Targets  
0  buying ring ordering amazon jeweler future fir...  
1  worth returning would pay postage ring cost ri...  
2  ring beautiful ring first shipment ring scratc...  
3  love shipping pace ring beautiful looking ring...  
4  ring gorgeous love ring would recommend anyone...  
